In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Please download and change path

Data: set of [ticker, headline, date]

File name: headlines.csv

Downloaded from: https://www.kaggle.com/code/shtrausslearning/news-sentiment-based-trading-strategy/input

In [2]:
# CHANGE TO YOUR PATH

In [3]:
#headlines_file_path = "/content/drive/MyDrive/LST/DS_SS2023/headlines_archive.zip"

In [4]:
#headlines_unzipped_file_path = "/content/drive/MyDrive/LST/DS_SS2023/headlines_archive_unzipped"

In [5]:
headlines_csv_path = "/content/drive/MyDrive/LST/DS_SS2023/headlines.csv"

# Install and Imports

## Installs

In [6]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install sentence-transformers==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-py3-none-any.whl size=86744 sha256=f352f82bfa9297e4811f5003f2456bc75b76d01af7d48e57301d20815641ac8a
  Stored in directory: /root/.cache/pip/wheels/2b/47/66/db4cf7cee20c1a4dcdcf947ce68b80b4893a979c70fcd2b3d2
Successfully built sentence-transformers


In [8]:
pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=402ab9371b1fbb39065023b29ecf3266dc826968f00fa73d7ba96134d43e9ac6
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=854be0af3835e533a9b5012356b82e63f1f5bc1e6eefbcfec807c76a2bd24b6a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing

In [9]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [10]:
# general imports
import os
import math
import json
import scipy
from scipy import spatial
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.manifold import TSNE
from bs4 import BeautifulSoup

In [14]:
# keybert imports
import keybert
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [15]:
# nltk/wordnet imports
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer as ps
from nltk.stem import WordNetLemmatizer

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [1]:
# data
import yfinance as yf

# Tools

## Sentence embeddings by SentenceTransformer

In [19]:
#from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

## KeyBERT

**Takes keywords from document level data**

documentation: https://pypi.org/project/keybert/

In [20]:
#keybert_model = KeyBERT()
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
keybert_model = KeyBERT(model=sentence_model)

In [21]:
# keywords of ngram length: up to 1 word
#test_keywords = keybert_model.extract_keywords(test_text, keyphrase_ngram_range=(1, 1), stop_words=None)

In [22]:
#test_keywords

In [23]:
# ngram length: up to 2 words
#test_keywords_len2 = keybert_model.extract_keywords(test_text, keyphrase_ngram_range=(1, 2), stop_words=None)

In [24]:
#test_keywords_len2

## Wordnet

**Wordnet synset similarities**

The wup_similarity method is short for Wu-Palmer Similarity, which is a scoring method based on how similar the word senses are and where the Synsets occur relative to each other in the hypernym tree.


---



Example:

score of "dog" and "cat" = 0.8571428571428571

score of "digitization" and "digital" = 0.14285714285714285

score of "digitization" and "technology" = 0.3

---



Comment: Good for word sense comparison. High similarity scores mean that the words are synonyms or are typologically similar, does not concern their co-occurences in context in particular.

### Initialise

In [25]:
lemmatizer = WordNetLemmatizer()

## Sentiment analysis

In [26]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")

In [27]:
sa_roberta_en = AutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
sa_roberta = pipeline("sentiment-analysis", model="soleimanian/financial-roberta-large-sentiment")
en_pipe = pipeline("text-classification", model=sa_roberta_en, tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


# Functions

## Get Industry-ticker sets

In [2]:
def get_tickers(raw_text):
  tickers_list = list()

  token_raw = word_tokenize(raw_text)
  for token in token_raw:
    if token.isupper() == True:
      if token.isalpha() == True:
        if len(token) >= 2:
          tickers_list.append(token)
          #print(token)

  tickers_list = list(set(tickers_list))

  return tickers_list

## Get Stock Price History Data

In [3]:
def get_prices_df(start, end, tickers):
  prices_df = pd.DataFrame()

  for ticker in tickers:
    ticker_yf = yf.Ticker(ticker)
    if prices_df.empty:
        prices_df = ticker_yf.history(start = start, end = end)
        prices_df['ticker']= ticker
    else:
        data_temp = ticker_yf.history(start = start, end = end)
        data_temp['ticker']= ticker
        prices_df = prices_df.append(data_temp)

  prices_df['ticker'] = prices_df['ticker'].replace({'META':'FB'})
  print(prices_df['ticker'].unique())

  return prices_df

## Get Headlines by Industry

In [4]:
def date_to_num(date_string):
  date_num = int()
  date_list = date_string.split("-")

  year = int(date_list[0])
  month = int(date_list[1])
  day = int(date_list[2])

  date_num = 365*year + 12*month + day

  return date_num

In [5]:
def get_headlines_by_industry(all_headlines_df, industry_dict, target_industry_name, target_start, target_end):
  industry_news_list = list()
  start = date_to_num(target_start)
  end = date_to_num(target_end)

  for index, row in all_headlines_df.iterrows():
    if row["ticker"] in industry_dict[target_industry_name]:
      head_date = date_to_num(row["date"])

      if head_date > start and head_date < end:
        append_list = [row["ticker"], row["headline"], row["date"]]
        industry_news_list.append(append_list)

  return industry_news_list

## Get Headlines by Company

In [6]:
def get_all_relevant_tickers(industry_tickers):
  all_tickers = list()

  for key, val in industry_tickers.items():
    #print(key, len(val))
    all_tickers += val
    #print("all tickers:", len(all_tickers))

  return all_tickers

In [33]:
def get_company_alt_name_list(ticker_name):
  ticker = yf.Ticker(ticker_name)
  alternative_name = ticker.info.get('shortName') #taken from https://aroussi.com/post/python-yahoo-finance
  company_alt_name_list = list()
  company_alt_name_list.append(ticker_name)
  company_alt_name_list.append(alternative_name)

  return company_alt_name_list

In [34]:
def get_company_headlines(all_headlines_df, target_tickers, target_start, target_end):
  company_headlines = dict()
  start = date_to_num(target_start)
  end = date_to_num(target_end)

  for now_ticker in target_tickers:
    company_alt_name_list = get_company_alt_name_list(now_ticker)
    heads = list()

    for index, row in all_headlines_df.iterrows():
      if row["ticker"] == now_ticker:
        head_date = date_to_num(row["date"])

        if head_date > start and head_date < end:
          append_list = [row["headline"], row["date"]]
          heads.append(append_list)

    print(now_ticker, len(heads))
    company_headlines[now_ticker] = [company_alt_name_list, heads]

  return company_headlines

## Get Industry Sentiment

In [35]:
def get_industry_info(industry_news, lemmatizer):
  kw_sentiment_dict = dict()
  kw_count_dict = dict()

  for id, head in enumerate(industry_news):
    filtered_keywords = list()
    keywords_raw = keybert_model.extract_keywords(head, keyphrase_ngram_range=(1,1), stop_words=None)
    #print("keywords_raw:", keywords_raw)

    for raw_kw in keywords_raw:
      kwsynset = wn.synsets(raw_kw[0])
      if len(kwsynset) >= 1:
        filtered_keywords.append(raw_kw[0])

    #print("filtered keywords:", keywords)

    sentiment_result = sa_roberta(head)

    #print("result:", sentiment_result)

    #print("roberta-large-financial-news-sentiment-en\n", en_pipe(head))
    #print("financial-roberta-large-sentiment\n", sa_roberta(head))

    sentiment = sentiment_result[0]["label"]
    confidence = sentiment_result[0]["score"]
    #print("sentiment:", sentiment)
    #print("keywords", keywords, "\n")

    if sentiment == "neutral":
      sentiment_score = 0.0001 * confidence
    elif sentiment == "positive":
      sentiment_score = 1 * confidence
    elif sentiment == "negative":
      sentiment_score = -1 * confidence

    kw_sentiment_dict, kw_count_dict = get_keyword_dicts(filtered_keywords, kw_sentiment_dict, kw_count_dict, sentiment_score, lemmatizer)

  return kw_sentiment_dict, kw_count_dict

In [36]:
def get_industry_score(kw_sentiment_dict, kw_count_dict):

  industry_score = sum(list(kw_sentiment_dict.values())) / sum(list(kw_count_dict.values()))

  return industry_score

In [37]:
def get_keyword_dicts(keywords, kw_sentiment_dict, kw_count_dict, sentiment_score, lemmatizer):

  for kw in keywords:
    kw_lemm = lemmatizer.lemmatize(kw)
    if kw_lemm in kw_sentiment_dict:
      kw_sentiment_dict[kw_lemm] += sentiment_score
    else:
      kw_sentiment_dict[kw_lemm] = sentiment_score

    if kw_lemm in kw_count_dict:
      kw_count_dict[kw_lemm] += 1
    else:
      kw_count_dict[kw_lemm] = 1

  return kw_sentiment_dict, kw_count_dict

## Get Company Sentiment

In [38]:
def get_company_subset(company_headlines, company_ticker, industry_keyword_list, company_name_skiplist):
  company_all_news_sentiment = dict()
  company_subset_sentiment = dict()

  company_all_news = company_headlines[company_ticker][1]

  for head, date in company_all_news:
    #print("\n")
    #print("headline:", head)
    keywords = keybert_model.extract_keywords(head, keyphrase_ngram_range=(1,1), stop_words=None)
    sentiment_result = sa_roberta(head)

    #print("roberta-large-financial-news-sentiment-en\n", en_pipe(head))
    #print("financial-roberta-large-sentiment\n", sa_roberta(head))

    sentiment = sentiment_result[0]["label"]
    confidence = sentiment_result[0]["score"]
    #print("sentiment:", sentiment)
    #print("keywords", keywords)

    if sentiment == "neutral":
      sentiment_score = 0.0001 * confidence
    elif sentiment == "positive":
      sentiment_score = 1 * confidence
    elif sentiment == "negative":
      sentiment_score = -1 * confidence

    company_all_news_sentiment[head] = sentiment_score

    for kw, score in keywords:        # direct keyword matching here, can add glove embeddings related words as TODO later as well
      #print(kw, score)
      kw_lemm = lemmatizer.lemmatize(kw)
      if kw_lemm in industry_keyword_list: # when company news is related to industry news keywords

        if kw.lower() == company_ticker:
          if head not in company_subset_sentiment:
              company_subset_sentiment[head] = sentiment_score

        elif kw.lower() not in company_name_skiplist:
          #print("kw \"", kw, "\" in dict with sentiment score:", industry_keyword_list[kw])
          if head not in company_subset_sentiment:
            company_subset_sentiment[head] = sentiment_score

  return company_all_news_sentiment, company_subset_sentiment

# Run

In [39]:
headlines_csv_df = pd.read_csv(headlines_csv_path)

In [40]:
industry_tickers = {"Internet Content & Information": [],   # 65 companies
                    "Entertainment": [],                    # 45 companies
                    "Travel Services": [],                  # 16 companies
                    }

In [41]:
industry_headlines = {"Internet Content & Information": [],
                      "Entertainment": [],
                      "Travel Services": [],
                      }

In [42]:
industry_company_sentiments = {"Internet Content & Information": [],
                               "Entertainment": [],
                               "Travel Services": [],
                               }

In [50]:
industry_sentiments = {"Internet Content & Information": [],
                        "Entertainment": [],
                        "Travel Services": [],
                        }

In [ ]:
INTERNET = "https://stockanalysis.com/stocks/industry/internet-content-and-information/"
ENTERTAINMENT = "https://stockanalysis.com/stocks/industry/entertainment/"
TRAVEL = "https://stockanalysis.com/stocks/industry/travel-services/"
def parse_tickers(url):
  # implemented using https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-the-tree
  # and https://www.crummy.com/software/BeautifulSoup/bs4/doc/#quick-start
  # and https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree
  result = {}
  soup = BeautifulSoup(url, 'html.parser')
  table = soup.find("main-table")
  tbody = table.find("tbody")
  rows = tbody.find_all("tr")
  for row in rows:
    symbol = row[1]
    company_name = row[2]
    market_cap = row[3]
    percent_change = row[4]
    volume = row[5]
    revenue = row[6]
    result[symbol] = {"company_name" : company_name, "market_cap":market_cap,
                      "percent_change" : percent_change, "volume" : volume, "revenue" : revenue}
  return result




In [43]:
# copied raw text of tickers by industries from https://stockanalysis.com/stocks/industry/


raw_industry1_text = "1	GOOGL	Alphabet Inc.	1,531.88B -2.20% 8,215,944	284.61B 2	META	Meta Platforms, Inc.	719.72B -1.16% 4,826,119	117.35B 3	BIDU	Baidu, Inc.	49.60B -1.14% 626,785	18.40B 4	SPOT	Spotify Technology S.A.	29.85B -1.56% 933,574	13.16B 5	DASH	DoorDash, Inc.	28.18B -0.66% 611,516	7.16B 6	PINS	Pinterest, Inc.	16.93B -1.77% 2,149,388	2.83B 7	SNAP	Snap Inc.	16.43B -3.12% 4,483,302	4.53B 8	TME	Tencent Music Entertainment Group	13.15B 1.33% 1,718,368	4.18B 9	TWLO	Twilio Inc.	11.95B -3.19% 906,172	3.96B 10	MTCH	Match Group, Inc.	11.43B -3.25% 2,117,009	3.18B 11	Z	Zillow Group, Inc.	11.07B -0.54% 399,736	2.40B 12	IAC	IAC Inc.	5.24B 0.56% 130,638	4.99B 13	ATHM	Autohome Inc.	3.50B -0.66% 164,846	- 14	WB	Weibo Corporation	3.33B -1.43% 146,929	1.77B 15	CARG	CarGurus, Inc.	2.44B -2.51% 354,680	1.46B 16	YELP	Yelp Inc.	2.44B -2.80% 72,373	1.23B 17	YY	JOYY Inc.	2.07B 1.16% 298,531	329.88M 18	GETY	Getty Images Holdings, Inc.	2.01B -1.57% 194,886	930.91M 19	MOMO	Hello Group Inc.	1.79B 0.90% 318,003	12.37B 20	SSTK	Shutterstock, Inc.	1.73B -0.29% 97,955	843.97M 21	OPRA	Opera Limited	1.64B 1.73% 353,634	346.50M 22	ANGI	Angi Inc.	1.62B -0.77% 239,987	1.85B 23	PERI	Perion Network Ltd.	1.55B -1.78% 106,556	660.09M 24	SEAT	Vivid Seats Inc.	1.40B -2.05% 79,004	630.57M 25	GENI	Genius Sports Limited	1.20B 0.70% 233,265	352.34M 26	KIND	Nextdoor Holdings, Inc.	1.06B -2.59% 229,864	211.54M 27	FVRR	Fiverr International Ltd.	1.02B -3.27% 261,398	338.64M 28	TTGT	TechTarget, Inc.	931.55M -3.63% 30,401	286.44M 29	MATH	Metalpha Technology Holding Limited	901.37M	-	2,745	5.30M 30	THRY	Thryv Holdings, Inc.	883.48M -1.13% 33,105	1.14B 31	TBLA	Taboola.com Ltd.	853.64M -1.37% 202,674	1.37B 32	ZH	Zhihu Inc.	690.32M 2.40% 964,612	561.27M 33	MAX	MediaAlpha, Inc.	572.80M -0.62% 15,880	428.10M 34	SMWB	Similarweb Ltd.	521.32M -0.74% 9,741	201.70M 35	TRVG	trivago N.V.	399.08M -1.69% 269,821	591.74M 36	DOYU	DouYu International Holdings Limited	326.17M -0.98% 302,219	989.18M 37	JFIN	Jiayin Group Inc.	312.18M -8.44% 75,428	565.12M 38	NN	NextNav Inc.	297.30M -3.83% 62,107	3.56M 39	OB	Outbrain Inc.	246.55M -0.31% 18,489	969.64M 40	EVER	EverQuote, Inc.	222.01M -3.86% 76,951	402.67M 41	TRUE	TrueCar, Inc.	185.98M -3.70% 59,551	154.97M 42	LTRPA	Liberty TripAdvisor Holdings, Inc.	179.53M 1.04% 4,515	1.60B 43	STBX	Starbox Group Holdings Ltd.	163.13M -0.99% 25,877	7.19M 44	BODY	The Beachbody Company, Inc.	152.85M 0.43% 272,678	638.18M 45	GRPN	Groupon, Inc.	135.18M 2.58% 330,935	567.38M 46	BZFD	BuzzFeed, Inc.	88.55M -4.10% 303,112	412.27M 47	SCOR	comScore, Inc.	81.34M -1.72% 11,692	374.02M 48	AREN	The Arena Group Holdings, Inc.	79.42M 2.29% 5,270	224.07M 49	CMCM	Cheetah Mobile Inc.	68.76M 1.42% 9,905	126.84M 50	EGLX	Enthusiast Gaming Holdings Inc.	68.30M -1.41% 30,347	143.88M 51	LIZI	Lizhi Inc.	49.79M -2.65% 173,962	334.72M 52	UPXI	Upexi, Inc.	40.98M 3.72% 28,985	78.06M 53	IZEA	IZEA Worldwide, Inc.	40.43M -4.49% 13,436	40.94M 54	FAZE	FaZe Holdings Inc.	35.30M 0.21% 27,900	66.77M 55	KRKR	36Kr Holdings Inc.	33.41M 2.25% 20,384	47.80M 56	ZDGE	Zedge, Inc.	31.54M 1.89% 10,467	27.98M 57	ASST	Asset Entities Inc.	25.68M -6.90% 194,059	278.18K 58	FENG	Phoenix New Media Limited	24.51M -3.35% 5,265	110.14M 59	LOV	Spark Networks SE	20.73M -0.01% 14,824	195.55M 60	LKCO	Luokung Technology Corp.	19.79M -1.57% 39,600	93.59M 61	SLGG	Super League Gaming, Inc.	17.39M -7.56% 1,128,511	19.23M 62	TC	TuanChe Limited	13.28M 3.41% 5,659	26.28M 63	LCFY	Locafy Limited	7.27M	-	1	- 64	ONFO	Onfolio Holdings, Inc.	6.29M -2.36% 20,833	3.19M 65	GROM	Grom Social Enterprises, Inc.	3.07M 0.08% 79,583	5.40M"
raw_industry2_text = "1	NFLX	Netflix, Inc.	191.53B -0.89% 1,183,261	31.91B 2	DIS	The Walt Disney Company	162.32B -1.05% 4,454,386	86.98B 3	WBD	Warner Bros. Discovery, Inc.	29.81B 0.37% 4,136,713	41.36B 4	LYV	Live Nation Entertainment, Inc.	20.45B -0.25% 133,282	18.01B 5	FWONA	Formula One Group	18.02B 0.79% 11,787	12.15B 6	FOXA	Fox Corporation	16.32B 0.24% 553,802	14.91B 7	SIRI	Sirius XM Holdings Inc.	14.43B -0.27% 2,214,063	8.96B 8	WMG	Warner Music Group Corp.	13.54B -0.98% 138,267	5.82B 9	NWSA	News Corporation	11.02B 0.39% 239,630	10.12B 10	PARA	Paramount Global	10.39B 0.70% 3,067,447	30.09B 11	ROKU	Roku, Inc.	9.36B -2.30% 1,327,813	3.13B 12	WWE	World Wrestling Entertainment, Inc.	7.53B 0.11% 74,162	1.26B 13	EDR	Endeavor Group Holdings, Inc.	6.68B 0.77% 269,294	5.39B 14	NXST	Nexstar Media Group, Inc.	5.84B 0.22% 42,531	5.26B 15	IQ	iQIYI, Inc.	5.09B 0.38% 2,076,703	4.38B 16	MSGS	Madison Square Garden Sports Corp.	4.21B 0.40% 11,015	935.73M 17	AMC	AMC Entertainment Holdings, Inc.	4.08B 0.11% 3,728,723	4.08B 18	MANU	Manchester United plc	3.76B 2.51% 344,834	701.59M 19	BOWL	Bowlero Corp.	2.02B 2.64% 751,343	1.09B 20	CNK	Cinemark Holdings, Inc.	2.00B 0.18% 357,207	2.60B 21	LGF.A	Lions Gate Entertainment Corp.	2.00B -1.14% 53,107	3.85B 22	MSGE	Madison Square Garden Entertainment Corp.	1.87B -9.02% 207,752	- 23	PLAY	Dave & Buster's Entertainment, Inc.	1.80B -0.17% 245,094	2.11B 24	IMAX	IMAX Corporation	952.59M 0.17% 55,594	327.72M 25	SPHR	Sphere Entertainment Co.	947.70M -2.29% 125,361	1.86B 26	HUYA	HUYA Inc.	921.22M 2.13% 848,324	1.27B 27	SBGI	Sinclair Broadcast Group, Inc.	813.83M -1.26% 163,849	3.41B 28	MCS	The Marcus Corporation	465.12M -0.27% 24,135	697.43M 29	AMCX	AMC Networks Inc.	461.58M -0.75% 76,246	3.10B 30	RSVR	Reservoir Media, Inc.	395.62M 2.17% 71,286	122.29M 31	LVO	LiveOne, Inc.	136.75M -1.26% 74,908	97.50M 32	GNUS	Genius Brands International, Inc.	95.87M -1.48% 121,662	75.05M 33	RDI	Reading International, Inc.	93.67M 0.76% 4,565	208.72M 34	HOFV	Hall of Fame Resort & Entertainment Company	56.23M 7.50% 22,840	16.99M 35	GAIA	Gaia, Inc.	51.76M 2.06% 204	79.85M 36	MMV	MultiMetaVerse Holdings Limited	45.49M -4.79% 75,144	12.23M 37	AGAE	Allied Gaming & Entertainment Inc.	40.39M	-	3,006	5.13M 38	CSSE	Chicken Soup for the Soul Entertainment, Inc.	35.04M -2.44% 68,074	333.20M 39	ANGH	Anghami Inc.	27.83M 0.94% 25,716	48.48M 40	DLPN	Dolphin Entertainment, Inc.	25.46M -2.21% 2,917	41.22M 41	BREA	Brera Holdings PLC	22.35M -1.28% 2,707	172.77K 42	CNVS	Cineverse Corp.	20.03M 1.40% 51,301	72.33M 43	KUKE	Kuke Music Holding Limited	17.75M -13.15% 26,542	16.52M 44	ADD	Color Star Technology Co., Ltd.	12.73M 1.14% 7,769	7.09M 45	CPOP"
raw_industry3_text = "1	BKNG	Booking Holdings Inc.	96.85B -0.59% 48,631	18.17B 2	ABNB	Airbnb, Inc.	79.24B -0.62% 1,149,329	8.71B 3	RCL	Royal Caribbean Cruises Ltd.	24.59B -0.29% 816,217	10.67B 4	TCOM	Trip.com Group Limited	22.66B 1.01% 956,415	3.66B 5	CCL	Carnival Corporation & plc	20.44B -1.29% 11,066,307	14.98B 6	EXPE	Expedia Group, Inc.	15.60B -1.48% 465,782	12.08B 7	NCLH	Norwegian Cruise Line Holdings Ltd.	8.12B -0.75% 2,453,274	6.14B 8	TNL	Travel + Leisure Co.	3.08B -0.57% 69,998	3.64B 9	MMYT	MakeMyTrip Limited	2.86B 0.97% 28,411	593.04M 10	TRIP	Tripadvisor, Inc.	2.30B -2.09% 383,123	1.60B 11	SABR	Sabre Corporation	1.05B -4.38% 2,218,515	2.69B 12	MOND	Mondee Holdings, Inc.	750.89M 1.72% 27,435	171.76M 13	LIND	Lindblad Expeditions Holdings, Inc.	550.09M -0.39% 29,659	497.05M 14	DESP	Despegar.com, Corp.	469.00M -0.42% 23,522	584.27M 15	TOUR	Tuniu Corporation	206.76M -0.60% 44,577	29.89M 16	YTRA	Yatra Online, Inc.	134.79M 3.37% 31,838	46.56M Stock Analysis Pro Unlimited access to all our financial data with up to 30 years of history. Stock Screener Filter, sort and analyze all stocks to find your next investment. Watchlists Keep track of your favorite stocks in real-time. Market Newsletter Market-moving news and events each morning in short, bullet-point format."

In [44]:
industry_tickers["Internet Content & Information"] = get_tickers(raw_industry1_text)
industry_tickers["Entertainment"] = get_tickers(raw_industry2_text)
industry_tickers["Travel Services"] = get_tickers(raw_industry3_text)

In [45]:
date_start = "2010-01-01"   # YYYY-MM-DD
date_end = "2019-12-31"     # YYYY-MM-DD

In [46]:
# Get headlines given industry and target period

for key in industry_headlines.keys():
  industry_headlines[key] = get_headlines_by_industry(headlines_csv_df, industry_tickers, key, date_start, date_end)

In [51]:
all_target_tickers = get_all_relevant_tickers(industry_tickers)
#print(len(all_target_tickers))
#print(all_target_tickers[:5])

# runtime approx 15 mins
company_headlines_dict = get_company_headlines(headlines_csv_df, all_target_tickers, date_start, date_end)

YY 62
EVER 4
SCOR 46
LIZI 0
YELP 155
TRUE 38
SE 5
IZEA 9
NN 0
AREN 0
PERI 28
GENI 0
GRPN 195
FAZE 0
BIDU 157
TWLO 51
BZFD 0
OPRA 2
EGLX 0
GROM 0
UPXI 0
LCFY 0
TBLA 0
ASST 0
FENG 23
ONFO 0
ATHM 11
ZH 0
TME 11
SMWB 0
ZDGE 4
SPOT 31
LOV 3
OB 2
MTCH 48
WB 30
SEAT 0
ANGI 83
TTGT 8
FVRR 1
SNAP 100
STBX 0
SSTK 14
DOYU 0
MATH 0
TRVG 7
LTRPA 3
META 0
GOOGL 5
KRKR 0
BODY 13
THRY 0
LKCO 2
GETY 0
SLGG 3
CARG 6
JFIN 0
KIND 0
IAC 30
TC 0
JOYY 0
MAX 0
CMCM 38
MOMO 27
DASH 0
PINS 5
GNUS 0
LYV 20
SPHR 0
AGAE 0
GAIA 2
MSGS 0
BOWL 0
IMAX 40
FWONA 25
ADD 0
AMCX 36
HOFV 0
PARA 0
MANU 11
LVO 0
PLC 0
RDI 4
PLAY 50
HUYA 16
EDR 3
WWE 45
MSGE 0
FOXA 11
NFLX 379
DIS 158
CNK 9
WMG 0
RSVR 0
CNVS 0
NWSA 12
ANGH 0
WBD 0
XM 1
MMV 0
CSSE 1
MCS 3
IQ 17
SIRI 72
DLPN 0
ROKU 68
AMC 56
CPOP 0
SBGI 54
BREA 0
NXST 30
KUKE 0
TRIP 75
CCL 76
BKNG 86
RCL 44
MOND 0
TNL 0
SABR 6
TOUR 6
TCOM 0
NCLH 30
EXPE 93
ABNB 0
DESP 3
MMYT 14
LIND 2
YTRA 4


In [52]:
# Industry sentiments

for industry in industry_sentiments.keys():
  print("industry:", industry)

  ind_head = [item[1] for item in industry_headlines[industry]]

  # I use this for testing
  # total runtime approx 15 mins on 200 entries per industry...
  ind_head = ind_head[:200]

  print("ind head:", len(ind_head))

  ind_kw_sentiment_dict, ind_kw_count_dict = get_industry_info(ind_head, lemmatizer)
  ind_sentiment = get_industry_score(ind_kw_sentiment_dict, ind_kw_count_dict)

  print("sentiment dict:", len(ind_kw_sentiment_dict), "keyword count:", len(ind_kw_count_dict))
  print("industry score:", ind_sentiment)

  industry_sentiments[industry] = [ind_sentiment, ind_kw_sentiment_dict, ind_kw_count_dict]

industry: Internet Content & Information
ind head: 200
sentiment dict: 162 keyword count: 162
industry score: -0.007563214185537882
industry: Entertainment
ind head: 200
sentiment dict: 191 keyword count: 191
industry score: 0.14840382082267303
industry: Travel Services
ind head: 200
sentiment dict: 155 keyword count: 155
industry score: 0.14693488958412468


In [57]:
# test run keywords

print("Example: Travel Services")
print("Inudstry sentiment score:", industry_sentiments["Travel Services"][0], "\n")

ts_test_keys = ["caribbean", "norwegian", "hurricane", "booking", "revenue"]

print("sentiment scores of some keywords:")

for key in ts_test_keys:
  total_sent = industry_sentiments["Travel Services"][1][key]
  total_count = industry_sentiments["Travel Services"][2][key]
  print(key, total_sent / total_count)

Example: Travel Services
Inudstry sentiment score: 0.14693488958412468 

sentiment scores of some keywords:
caribbean 0.09681763432242653
norwegian 0.5983229577541351
hurricane -0.9988762736320496
booking 0.5998358408610026
revenue 0.1902317869581282


In [ ]:
# Company final sentiments

for industry in industry_tickers.keys():
  target_tickers_list = industry_tickers[industry]
  now_industry_sentiment = industry_sentiments[industry][0]
  now_industry_company_sentiment = dict()

  print("Industry:", industry)
  print("Industry sentiment:", now_industry_sentiment)

  for now_ticker in target_tickers_list:
    print("company:", now_ticker)

    # TODO: company_name_skip_list = list of all company names (conventional names & tickers) that are not the same company as "now_ticker"
    # ref: use get_company_alt_name_list() after you have that list
    company_name_skip_list = []

    now_company_allnews_sent, now_company_subset_sent = get_company_subset(company_headlines_dict, now_ticker, industry_sentiments[industry][1], company_name_skip_list)

    if len(now_company_allnews_sent) >= 1:
      now_company_allnews_score = sum(list(now_company_allnews_sent.values())) / len(now_company_allnews_sent)
    else:
      now_company_allnews_score = 0

    if len(now_company_subset_sent) >= 1:
      now_company_subset_score = sum(list(now_company_subset_sent.values())) / len(now_company_subset_sent)
    else:
      now_company_subset_score = 0

    print("company_allnews_sentiment:", now_company_allnews_sent)
    print("company_subset_sentiment:", now_company_subset_sent)

    print("company score (all news concerned):", now_company_allnews_score)
    print("company score (as influenced by industry):", now_company_subset_score)

    ###################################

    now_company_rel_score = (now_company_allnews_score + now_company_subset_score) / 2
    now_company_final_score = now_industry_sentiment + now_company_rel_score

    now_industry_company_sentiment[now_ticker] = now_company_final_score

    print("company relevant [(now_company_allnews_score + now_company_subset_score) /2]: ", now_company_rel_score)
    print("company final score: (industry_sentiment + now_company_rel_score)", now_company_final_score)

  industry_company_sentiments[industry] = now_industry_company_sentiment


Industry: Internet Content & Information
Industry sentiment: -0.007563214185537882
company: YY
company_allnews_sentiment: {"YY has delivered a solid, if unspectacular, IPO. Shares opened at $10.69 and have since risen to $11.74, good for an 11.8% gain. YY's IPO price of $10.50 was at the low end of a $10.50-$12.50 range. (previous) (F-1)": 0.997772753238678, 'YY (NASDAQ:YY): Q4 Non-GAAP ADS of $4.74. Revenue of $42.8M (+136% Y/Y). Shares  -5.3% AH. (PR)': 0.9974362254142761, "Chinese Internet stocks jump in response to Qihoo's Q1 beat and very strong Q2 guidance, highlighted by ramping search ad sales and soaring online game revenue. YY +6.5%. RENN +6.8%. NTES +5.5%. BIDU +2.4%. PWRD +3.2%. SOHU +3%. SINA +2.3%.": 0.998127281665802, 'Chinese Internet and solar stocks fall hard after growing banking sector liquidity worries contribute to a 5.3% drop for the Shanghai Composite. Notable decliners: YY -13%. DANG -9%. GA -8.9%. PWRD -7.2%. YOKU -6.6%. SOHU -5.8%. NQ -5.6%. CSIQ -8.4%. TSL -

## Final sentiments of companies in each industry

In [ ]:
industry_company_sentiments["Internet Content & Information"]

In [ ]:
industry_company_sentiments["Entertainment"]

In [ ]:
industry_company_sentiments["Travel Services"]